# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [2]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

сразу можно заметить, что кредит в срок закрывал вообще мало кто - стоит ожидать в любом случае довольно малых шансов закрытия в срок от всех категорий клиентов

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


значения в столбах **days_employed** и **total_income** для удобства просмотра можно будет перевести в int. все столбцы названы верно с точки зрения хорошего стиля

In [4]:
data.isna().sum()
# посмотрим в каких столбцах имеются пропуски

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

пропуски зафиксированы в столбцах с уровнем дохода и с данными о времени трудоустройства. вполне логично предположить, что у речь идет о клиентах вообще без трудового стажа, которым и в качестве дохода было указывать нечего. заменим такие значения на 0 в обоих таблицах

In [5]:
data_none = data[data['days_employed'].isna() == True]
display(data_none)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [6]:
data_none['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [7]:
del data['days_employed']

In [8]:
display(data.groupby('income_type')['total_income'].mean())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [9]:
mean_income = data.groupby('income_type')['total_income'].transform('mean')
data['total_income'].fillna(mean_income, inplace=True)
data['total_income'] = data['total_income'].astype('int')                      # для удобства просмотра переведём float в int
display(data.head(30))

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [10]:
data.isna().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Теперь проверим столбец **dob_years**

In [11]:
sorted(data['dob_years'].unique())

[0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

In [12]:
display(data[data['dob_years'] == 0])

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...
19829,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,161380,жилье
20462,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20577,0,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21179,2,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


Какой-то закономерности относительно данных в других столбцах тут нельзя рассмотреть, но 101 строка - достаточно немалый срез, чтобы его просто удалять. Поскольку финальное исследование, которое мы делаем - не зависит от возраста клиента, предварительно данные строки можно оставить и списать их на техническую ошибку при заполнении данных, как мне кажется, и если потребуется в дальнейшем исследование именно по возрастному критерию - запросить данные по этим пропущенным позициям, если это будет возможно. Однако поскольку данные в этом столбце в отличие от days_employed в своей общей массе выглядят вполне адекватно, по аналогии с заполнением среднего уровня дохода я решил попробовать заполнить их, опираясь на столбец income_type. Конечно "безработный" или "сотрудник" - могут быть почти любого возраста, но например для таких клиентов как "студент" или "пенсионер" вывод их среднего возраста может быть вполне хорошим решением

In [13]:
age_classification = data.groupby('income_type')['dob_years'].mean()
display(age_classification)

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.636737
компаньон          39.697542
пенсионер          59.063019
предприниматель    42.500000
сотрудник          39.821027
студент            22.000000
Name: dob_years, dtype: float64

в своём большинстве соответствия выглядят даже вполне адекватно - студенты действительно чуть старше 20, а пенсионерам около 60. думаю можно попытаться заполнить 0 в возрастах полагаясь именно на эту таблицу

In [14]:
def fullfill_zeros(data):
    global age_classification
    if data['dob_years'] == 0:
        return age_classification[data['income_type']]
    else:
        return data['dob_years']


data['dob_years'] = data.apply(fullfill_zeros, axis=1)

In [15]:
data['dob_years'] = data['dob_years'].astype('int') 
# так как годы должны быть целыми числами, хоть и были получены усредненными значениями, переведем float в int

In [16]:
print(data['dob_years'].min())    # проверим теперь минимальный и максимальный возраст
data['dob_years'].max()

19


75

In [17]:
data.loc[1040, 'dob_years']        # возьмем в качестве проверки одну из строк, где ранее был возраст = 0

39

Как видно, замена была проведена успешно, и хоть и с натянутой точностью, но технические ошибки при заполнении столбца с возрастом клиентов были устранены, и вместо 0 вставлены усредненные значения из общей статистики

Теперь проверим данные в столбце о кол-ве детей **children**:

In [18]:
data['children'].max()

20

In [19]:
data['children'].min()

-1

In [20]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Число детей может быть = 0 , но не отрицательным, так же довольно сомнительно выглядит 20 детей - скорее всего имели ввиду 2 

In [21]:
data['children']=data['children'].replace(-1,1)
data['children']=data['children'].replace(20,2)

In [22]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверим значения в столбце **education**:

In [23]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [24]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

видно что столбец **education** содержит дубликаты, которые далее стоит обработать

Проверим значения в столбце **family_status** и **family_status_id**

In [25]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [26]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

хоть кол-во уникальных **family_status_id** совпадает с кол-вом уникальных **family_status** - однако глобально нам не нужно целых 5 категорий в данных столбцах. можно лишь просто разделить всё в дальнейшем на *не в браке* и *в браке* - так как все 5 уникальных значений по сути являются дубликатами этих двух. пропущенных значений в данных столбцах нету

In [27]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [28]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [29]:
data = data[data['gender'] != 'XNA'].reset_index(drop=True)

в столбце **gender** было одно значение, видимо записанное по ошибке. при его удалении мы потеряли всего одну строку, так что эту неточность можно не обсуждать отдельно с заказчиком, так как сильно на итоговый результат отсутствие данной одной строки не повлияет

In [30]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

в целом, все значения в **income_type** достаточно уникальные по смыслу, но для наглядности исследования можно будет разделить на работающих и безработных клиентов

In [31]:
data['debt'].unique()

array([0, 1])

к столбцу **debt** претензий нету

In [32]:
data['total_income'].min()

20667

правдоподобность значений доходов мы не можем оценить, однако хотя бы убедились, что среди них нет отрицательных значений. так что столбцу **total_income** можно довериться

In [33]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

на первый взгляд можно различить здесь взятие средств для оплаты машины, недвижимости или обрзования, поэтому для более наглядного анализа значения данного столбца **purpose** можно при помощи лемматиизации сгруппировать

## Шаг 2. Предобработка данных

### Обработка пропусков

пропуски были заполнены выше сразу после их обнаружения

### Замена типа данных

столбцы с типом данных float были заменены на int выше

### Обработка дубликатов

Дубликаты стоит обработать в столбце **education**. Выше при выводе уникальных значений было явно показано, что проблема в том, что значения в данном стобце не приведены к одному регистру. Приведём их все к нижнему регистру и проверим уникальные значения снова

In [34]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [35]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [36]:
data['education_id'].value_counts()

1    15233
0     5260
2      743
3      282
4        6
Name: education_id, dtype: int64

Как и ожидалось, после удаления всех дубликатов столбца **education** выяснилось, что в столбце **education_id** каждой степени образования соответствует определённый идентификатор. Чтобы разгрузить общий датафрейм, но и не забыть чему равно каждое из данных значений - создадим словарь с парами ключ - **id** и значение - **уровень образования** , а столбец **education** удалим, так как пользоваться идентификаторами проще, чем полными названиями величин, которым они соответствуют. 

In [37]:
# к размышлению
data.groupby('education_id')['education'].first()

education_id
0                 высшее
1                среднее
2    неоконченное высшее
3              начальное
4         ученая степень
Name: education, dtype: object

In [38]:
education_data = data[['education', 'education_id']].drop_duplicates()
education_dict = education_data.groupby('education_id')['education'].apply(list).to_dict()
education_dict

{0: ['высшее'],
 1: ['среднее'],
 2: ['неоконченное высшее'],
 3: ['начальное'],
 4: ['ученая степень']}

так как мы сохранили данные о том, чему соответствуют id - идентификаторы, можно убрать столбец education для разгрузки памяти

In [39]:
del data['education']

Обработка столбца **family status**

In [40]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [41]:
data['family_status_id'].value_counts()

0    12380
1     4176
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [42]:
family_status_data = data[['family_status', 'family_status_id']].drop_duplicates()
family_id_dict = family_status_data.groupby('family_status_id')['family_status'].apply(list).to_dict()
family_id_dict

{0: ['женат / замужем'],
 1: ['гражданский брак'],
 2: ['вдовец / вдова'],
 3: ['в разводе'],
 4: ['Не женат / не замужем']}

теперь расшифровка столбца записана в словарь, и столбец family_status просто дублирует данный словарь и занимает место - можно удалить его

In [43]:
del data['family_status']

In [44]:
data.duplicated().sum()

71

In [45]:
data = data.drop_duplicates().reset_index(drop=True)

In [46]:
data.duplicated().sum()     # строки-дубликаты успешно удалены

0

### Лемматизация

как обсуждалось выше, лемматизацию можно применить к столбцу **purpose** , так как глобально там описывается лишь трата денег на *жилье*, *образование* , *автомобиль* или *свадьба*

In [47]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [48]:
from pymystem3 import Mystem
m = Mystem()

purposes = ' '.join(list(data['purpose'].unique()))         # создание текстовой строки из списка уникальных значений
lemmas = m.lemmatize(purposes)

unique_lemmas = []                                          # для наглядности выведем только уникальные значения
for lemma in lemmas:
    if lemma not in unique_lemmas:
        unique_lemmas.append(lemma)

sorted(unique_lemmas)

['\n',
 ' ',
 'автомобиль',
 'высокий',
 'для',
 'дополнительный',
 'жилой',
 'жилье',
 'заниматься',
 'коммерческий',
 'на',
 'недвижимость',
 'образование',
 'операция',
 'подержать',
 'покупка',
 'получение',
 'приобретение',
 'проведение',
 'профильный',
 'ремонт',
 'с',
 'свадьба',
 'свой',
 'сдача',
 'сделка',
 'семья',
 'со',
 'собственный',
 'строительство',
 'сыграть']

на основе полученного списка напишем функцию, которая создаст новый столбец, опираясь на ключевые слова из старых значений

In [49]:
def purpose_def(purpose):
    lemma = m.lemmatize(purpose)
    if 'автомобиль' in lemma:
        return 'автомобиль'
    elif ('жилье' in lemma
        or 'жилой' in lemma
        or 'недвижимость' in lemma):
        return 'недвижимость'
    elif 'свадьба' in lemma:
        return 'свадба'
    elif 'образование' in lemma:
        return 'образование'
    else:
        return 'не удалось определить категорию'


data['purpose_category'] = data['purpose'].apply(purpose_def)
display(data)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,свадба
...,...,...,...,...,...,...,...,...,...,...
21448,1,43,1,1,F,компаньон,0,224791,операции с жильем,недвижимость
21449,0,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21450,1,38,1,1,M,сотрудник,1,89672,недвижимость,недвижимость
21451,3,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


In [50]:
data['purpose_category'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
свадба           2324
Name: purpose_category, dtype: int64

как видно, функция сработала верно, плюс не вывела значений "не удалось определить категорию" - можем удалить старый столбец

In [51]:
del data['purpose']
display(data)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category
0,1,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,32,1,0,M,сотрудник,0,267628,образование
4,0,53,1,1,F,пенсионер,0,158616,свадба
...,...,...,...,...,...,...,...,...,...
21448,1,43,1,1,F,компаньон,0,224791,недвижимость
21449,0,67,1,0,F,пенсионер,0,155999,автомобиль
21450,1,38,1,1,M,сотрудник,1,89672,недвижимость
21451,3,38,1,0,M,сотрудник,1,244093,автомобиль


### Категоризация данных

в исследуемом наборе данных можно разбить некоторые столбцы на категории для удобства анализа. например столбец **children** поделить на семьи *бездетные* , *обычное число детей* и *многодетные*. Так же большой набор численных данных в столбцах **total_income** и **dob_years** можно вполне разбить на категории для удобства анализа

разделим семьи на многодетные (3 и больше детей) , обычное число детей (1-2) и бездетные (0):

In [52]:
def children_number(number):
    if number == 0:
        return 'бездетные'
    elif 1 <= number < 3:
        return 'обычное число детей'
    elif number >= 3:
        return 'многодетные'
    else:
        return 'не обработано'
    
    
data['children_category'] = data['children'].apply(children_number)
display(data)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category,children_category
0,1,42,0,0,F,сотрудник,0,253875,недвижимость,обычное число детей
1,1,36,1,0,F,сотрудник,0,112080,автомобиль,обычное число детей
2,0,33,1,0,M,сотрудник,0,145885,недвижимость,бездетные
3,3,32,1,0,M,сотрудник,0,267628,образование,многодетные
4,0,53,1,1,F,пенсионер,0,158616,свадба,бездетные
...,...,...,...,...,...,...,...,...,...,...
21448,1,43,1,1,F,компаньон,0,224791,недвижимость,обычное число детей
21449,0,67,1,0,F,пенсионер,0,155999,автомобиль,бездетные
21450,1,38,1,1,M,сотрудник,1,89672,недвижимость,обычное число детей
21451,3,38,1,0,M,сотрудник,1,244093,автомобиль,многодетные


In [53]:
data['children_category'].value_counts()

бездетные              14090
обычное число детей     6983
многодетные              380
Name: children_category, dtype: int64

как видно, все значения удалось обработать и категоризировать. теперь более наглядно видно распределение по кол-ву детей у клиентов банка. можно удалить старый столбец, так как точное число детей в данном случае уже не сильно важный фактор

In [54]:
del data['children']

следующим столбцом обработаем **dob_years** - разделим так же его на категории. пусть меньше 25 будут молодые, от 26 до 50 клиенты среднего возраста, а старше - пожилые

In [55]:
data['age_category'] = pd.qcut(data['dob_years'], q=4, precision=0)   
# разобьем все значения возраста клиентов на 4 группы

In [56]:
display(data.head(10))

,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category,children_category,age_category
0,42,0,0,F,сотрудник,0,253875,недвижимость,обычное число детей,"(33, 42]"
1,36,1,0,F,сотрудник,0,112080,автомобиль,обычное число детей,"(33, 42]"
2,33,1,0,M,сотрудник,0,145885,недвижимость,бездетные,"(18, 33]"
3,32,1,0,M,сотрудник,0,267628,образование,многодетные,"(18, 33]"
4,53,1,1,F,пенсионер,0,158616,свадба,бездетные,"(42, 53]"
5,27,0,1,M,компаньон,0,255763,недвижимость,бездетные,"(18, 33]"
6,43,0,0,F,компаньон,0,240525,недвижимость,бездетные,"(42, 53]"
7,50,1,0,M,сотрудник,0,135823,образование,бездетные,"(42, 53]"
8,35,0,1,F,сотрудник,0,95856,свадба,обычное число детей,"(33, 42]"
9,41,1,0,M,сотрудник,0,144425,недвижимость,бездетные,"(33, 42]"


In [57]:
data['age_category'].value_counts()

(42, 53]    5448
(33, 42]    5365
(18, 33]    5365
(53, 75]    5275
Name: age_category, dtype: int64

Для столбца **total_income** попробуем выполнить подобную процедуру и разбить всё на 4 интервала:

In [58]:
data['income_category'] = pd.qcut(data['total_income'], q=4, precision=0)
display(data)

,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category,children_category,age_category,income_category
0,42,0,0,F,сотрудник,0,253875,недвижимость,обычное число детей,"(33, 42]","(202417, 2265604]"
1,36,1,0,F,сотрудник,0,112080,автомобиль,обычное число детей,"(33, 42]","(107620, 151876]"
2,33,1,0,M,сотрудник,0,145885,недвижимость,бездетные,"(18, 33]","(107620, 151876]"
3,32,1,0,M,сотрудник,0,267628,образование,многодетные,"(18, 33]","(202417, 2265604]"
4,53,1,1,F,пенсионер,0,158616,свадба,бездетные,"(42, 53]","(151876, 202417]"
...,...,...,...,...,...,...,...,...,...,...,...
21448,43,1,1,F,компаньон,0,224791,недвижимость,обычное число детей,"(42, 53]","(202417, 2265604]"
21449,67,1,0,F,пенсионер,0,155999,автомобиль,бездетные,"(53, 75]","(151876, 202417]"
21450,38,1,1,M,сотрудник,1,89672,недвижимость,обычное число детей,"(33, 42]","(20666, 107620]"
21451,38,1,0,M,сотрудник,1,244093,автомобиль,многодетные,"(33, 42]","(202417, 2265604]"


In [59]:
data['income_category'].value_counts()

(151876, 202417]     5818
(20666, 107620]      5364
(107620, 151876]     5363
(202417, 2265604]    4908
Name: income_category, dtype: int64

In [60]:
# распечатаем финальный вид данных для анализа в следующих пунктах:
display(data)

,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category,children_category,age_category,income_category
0,42,0,0,F,сотрудник,0,253875,недвижимость,обычное число детей,"(33, 42]","(202417, 2265604]"
1,36,1,0,F,сотрудник,0,112080,автомобиль,обычное число детей,"(33, 42]","(107620, 151876]"
2,33,1,0,M,сотрудник,0,145885,недвижимость,бездетные,"(18, 33]","(107620, 151876]"
3,32,1,0,M,сотрудник,0,267628,образование,многодетные,"(18, 33]","(202417, 2265604]"
4,53,1,1,F,пенсионер,0,158616,свадба,бездетные,"(42, 53]","(151876, 202417]"
...,...,...,...,...,...,...,...,...,...,...,...
21448,43,1,1,F,компаньон,0,224791,недвижимость,обычное число детей,"(42, 53]","(202417, 2265604]"
21449,67,1,0,F,пенсионер,0,155999,автомобиль,бездетные,"(53, 75]","(151876, 202417]"
21450,38,1,1,M,сотрудник,1,89672,недвижимость,обычное число детей,"(33, 42]","(20666, 107620]"
21451,38,1,0,M,сотрудник,1,244093,автомобиль,многодетные,"(33, 42]","(202417, 2265604]"


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [61]:
display(data.groupby('children_category')['debt'].mean()*100)

children_category
бездетные              7.544358
многодетные            8.157895
обычное число детей    9.265359
Name: debt, dtype: float64

**Вывод** : если наличие задолженности по кредиту - это единица в соответствующем столбце **debt**, то получается чем меньше процент в данной таблице, тем меньше шансов её встретить у соответствующего клиента задолженность. Получается - самая маленькая задолженность у клиентов без детей (около 7.5 %) , самая большая у семей с обчным числом детей в 1-2 ребёнка (больше 9 %) - и средний показатель между ними это многодетные семьи с более чем 2 детьми (3-5 детей в данном датафрейме)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [62]:
display(data.groupby('family_status_id')['debt'].mean()*100)

family_status_id
0    7.545182
1    9.349398
2    6.569343
3    7.112971
4    9.750890
Name: debt, dtype: float64

In [63]:
family_id_dict

{0: ['женат / замужем'],
 1: ['гражданский брак'],
 2: ['вдовец / вдова'],
 3: ['в разводе'],
 4: ['Не женат / не замужем']}

**Вывод** : меньше всего шанс встретить задолженность у вдов/вдовцов - 6.5 % , большие показатели у женатых/замужних или наоборот разведенных клиентов - чуть больше 7%. Самый большой процент закрытия долга не в назначенный срок у клиентов в гражданском браке и еще не вступивших в брак - больше 9%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [64]:
display(data.groupby('income_category')['debt'].mean()*100)

income_category
(20666, 107620]      7.960477
(107620, 151876]     8.856983
(151876, 202417]     8.490890
(202417, 2265604]    7.029340
Name: debt, dtype: float64

**Вывод**: зависимось примерно параболическая, если говорить о вероятности закрытия в срок кредита в зависимости от дохода. У клиентов, доход которых был при разбиении в группе с самым низким доходом (20 000 - 107 000) и наоборот самым высоким (202 000 - 2 265 000) - вероятность просрочки около 7 - 8 процентов. Клиенты же со средним доходом - от 107 000 до 202 000 имели уже шансы невыплаты побольше - 8 - 9 %

- Как разные цели кредита влияют на его возврат в срок?

In [65]:
display(data.groupby('purpose_category')['debt'].mean()*100)

purpose_category
автомобиль      9.359034
недвижимость    7.234043
образование     9.220035
свадба          8.003442
Name: debt, dtype: float64

**Вывод** выплата кредита в зависимости от цели трат средств с него показала - вероятнее всего кредит отдадут вовремя, если брали его на покупку недвижимости (невыплаты в срок всего 7.23 %) , и вероятнее всего получить просрочку по выплате, если будет идти речь о покупке машины (шанс невыплаты 9.35 %)

In [66]:
display(data.groupby('education_id')['debt'].mean()*100)

education_id
0     5.295238
1     8.990245
2     9.152086
3    10.992908
4     0.000000
Name: debt, dtype: float64

In [67]:
education_dict

{0: ['высшее'],
 1: ['среднее'],
 2: ['неоконченное высшее'],
 3: ['начальное'],
 4: ['ученая степень']}

Поскольку ранее разбивались на категории так же степени образования - по ним так же можно сделать анализ. Тут уже есть более резонансные выводы - клиенты с учёной степенью закрывали долг вообще всегда в назначенный срок, а вот у клиентов с начальным образованием/средним/неоконченным высшим - такой шанс незакрытия в сроки уже 9-10 %. Средний показатель в 5 % занимают клиенты с высшим образованием

Чтобы не пропадали зря оставшиеся без внимания столбцы, а так же проявил себя pivot_table, можем посмотреть с какой вероятностью погасят в срок кредит мужчины и женщины в зависимости от их возраста:

In [68]:
data_pivot = data.pivot_table(index=['gender'], columns='age_category', values = 'debt')
display(data_pivot)

age_category,"(18, 33]","(33, 42]","(42, 53]","(53, 75]"
gender,,,,
F,0.101884,0.078227,0.061147,0.046675
M,0.119964,0.098352,0.098744,0.084496


по данной таблице можем сказать, что вероятнее всего задолженность по кредиту получит молодой мужчина в из возрастной группы 18-33 года (12%), и менее вероятно задолженность будет у пожилой женщины с возрастом 53-75 лет (около 4.5%)

## Шаг 4. Общий вывод

Желательно чтобы потенциальный заёмщик был бездетным, так же был вдовцом/вдовой, имел высокий заработок, высшее образование или даже учёную степень, а так же чтобы его целью займа была покупка недвижимости. Так же желательно, чтобы это была женщина, так как из общей статистики для всех возрастных групп именно у мужчин чаще наблюдается задолженность.
Хуже всего давать займ семьям с 1-2 детьми, раведенным или состоящим в гражданском браке, с средним уровнем дохода и рассматривающим трату средств на автомобиль или образование